In [126]:
# MueLu
# ./MueLu_Driver.exe --matrixType=Laplace3D
lambdas = [
    1.00014,
    1.28415,
    1.45419,
    1.5712,
    1.65446,
    1.71466,
    1.75886,
    1.79194,
    1.81727,
    1.83715
]
N = len(lambdas)

Lets assume that $\lambda_k$ monotonically increase and obey the following law:
$$\lambda_{max} - \lambda_k = bc^k$$
or
$$ \lambda_k = \lambda_{max} - bc^k.$$
This could be seen as a function (for discrete values)
$$ y = a - bc^x. $$ 

We would like to run a nonlinear regression on it, but it's hard in this form. Lets do some transformations.

First, lets get rid of $a$ by doing differences:
$$ y(x^{k+1}) - y(x^k) = bc^{x^k}(1-c) $$
and if we define $Y(x) = y(x+1) - y(x)$, we could write
$$ Y(x) = b(1-c)c^x. $$
Lets now say $B = b(1-c)$, then
$$ Y = Bc^x $$
which is amenable to logarithmic transformation
$$ ln(Y) = ln(B) + xln(c), $$
which we can run a linear regression on.


In [134]:
first = 2

from scipy import polyfit
import math
ar = []
br = []
for n in range(first,N):
    Y = [lambdas[k+1] - lambdas[k] for k in range(0, n)]
    X = [k for k in range(0, n)]
    lnY = [math.log(y) for y in Y]
    (ar_, br_) = polyfit(X, lnY, 1)
    ar.append(ar_)
    br.append(br_)
    # for i in range(len(X)):
    #     print(lnY[i] - (ar*X[i] + br))

In [128]:
import math
B = [math.exp(br_)   for br_ in br] 
c = [math.exp(ar_)   for ar_ in ar]
b = [B[k]/(1-c[k])   for k in range(0,len(B))]
print('Convergence ratio estimates:')
['{:.3f}'.format(i) for i in c]

Convergence ratio estimates:


['0.599', '0.642', '0.667', '0.683', '0.695', '0.704', '0.714', '0.722']

$ \lambda_{max} \approx \lambda_k + bc^k $, right? :)

In [136]:
print('lambda_max estimates')
lambda_max = lambdas[first] + b[0]*math.pow(c[0],first)
print('%1d its: %.2f' % (first+1, lambda_max))
for n in range(first+1,N):
    new_lambda_max = lambdas[n] + b[n-first]*math.pow(c[n-first],n)
    print('%1d its: %.2f [%4.2f%%]' % (n+1, new_lambda_max, 100*(new_lambda_max/lambda_max - 1.0)))
    lambda_max = new_lambda_max

lambda_max estimates
3 its: 1.71
4 its: 1.78 [3.99%]
5 its: 1.81 [2.18%]
6 its: 1.84 [1.29%]
7 its: 1.85 [0.85%]
8 its: 1.87 [0.63%]
9 its: 1.87 [0.51%]
10 its: 1.88 [0.44%]
